In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import linprog
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from scipy.spatial import HalfspaceIntersection, ConvexHull
import itertools

#### The Lemke-Howson algorithm, a deeper look

Here I had a lot of trouble understanding the algorithm, but found another great resource at https://cnl.gmu.edu/TAVRI/research/LemkeHowson.pdf, which was more intuitive, and what I follow for this section.

Here $U_1$ and $U_2$ are $(n,m)$ matricies representing the payoffs of both players. $s_1$ and $s_2$ are the strategies (vector of probabilities, chance of taking each action). $v_1$ and $v_2$ are the best each player can do, given the other player's strategy. Finally, $r_1$ and $r_2$ are vectors representing the gap between the reward for each action and the best action, given the other player's strategy. 

At equilibrium we have:

$$
\begin{align*}
\text{} \quad U_1\cdot s_2 + r_1 & = v_1 \\
\text{} \quad U_2^T\cdot s_1 + r_2 & = v_2 \\
\text{} \quad s_1\cdot r_1 & = 0 \\
\text{} \quad s_2\cdot r_2 & = 0 \\
\text{} \quad \sum s_1 & = 1\\
\text{} \quad \sum s_2 & = 1
\end{align*}
$$

The first two equations state what we had above, that the benefit of each action plus some number equals the benefit of the best action. The third and fourth lines state that an action either has 0 probability, or it is the best action (or tied-best). The fifth and sixth state that the strategies are probabilities. We also need the condition that all variables $\geq 0$, but I have left that out for now. 

We can simplify this further by dividing the first two lines by $v_1$ and $v_2$ respectively:

$$
\begin{align*}
\text{} \quad U_1\cdot s_2/v_1 + r_1/v_1 & = 1 \\
\text{} \quad U_2^T\cdot s_1/v_2 + r_2/v_2 & = 1 \\
\text{} \quad s_1\cdot r_1 & = 0 \\
\text{} \quad s_2\cdot r_2 & = 0 \\
\text{} \quad \sum s_1 & = 1\\
\text{} \quad \sum s_2 & = 1 \\
\end{align*}
$$

Defining $s_1^\prime=s_1/v_2$, $s_2^\prime=s_2/v_1$, $r_1^\prime=r_1/v_1$, and $r_2^\prime=r_2/v_2$ we can rewrite the equations above and get:

$$
\begin{align*}
\text{} \quad U_1\cdot s_2^\prime + r_1^\prime & = 1 \\
\text{} \quad U_2^T\cdot s_1^\prime + r_2^\prime & = 1 \\
\text{} \quad (v_2s_1^\prime)\cdot (v_1r_1^\prime) & = 0 \\
\text{} \quad (v_1s_2^\prime)\cdot (v_2r_2^\prime) & = 0 \\
\text{} \quad \sum s_1^\prime & = 1/v_2 \\
\text{} \quad \sum s_2^\prime & = 1/v_1 \\
\end{align*}
$$

In the third and fourth lines the constants don't matter, so we can remove them (we also do this for the condition that all variables are $\geq 0$):

$$
\begin{align*}
\text{} \quad U_1\cdot s_2^\prime + r_1^\prime & = 1 \\
\text{} \quad U_2^T\cdot s_1^\prime + r_2^\prime & = 1 \\
\text{} \quad s_1^\prime\cdot r_1^\prime & = 0 \\
\text{} \quad s_2^\prime\cdot r_2^\prime & = 0 \\
\text{} \quad \sum s_1^\prime & = 1/v_2 \\
\text{} \quad \sum s_2^\prime & = 1/v_1 \\
\end{align*}
$$

We can remove the last two equations as the utility (and equilibrium) don't depend on any scaling of the utility function. I.e., say $v_1=c_1v_1^\text{truth}$. As we are allowed any $c_1$ we can just use $c_1=1/(v_1^\text{truth}\sum s_2^\prime)$ to get $v_1=1/(\sum s_2^\prime)$. 

$$
\begin{align*}
\text{} \quad U_1\cdot s_2^\prime + r_1^\prime & = 1 \\
\text{} \quad U_2^T\cdot s_1^\prime + r_2^\prime & = 1 \\
\text{} \quad s_1^\prime\cdot r_1^\prime & = 0 \\
\text{} \quad s_2^\prime\cdot r_2^\prime & = 0 \\
\end{align*}
$$

The first equation above describes $n$ different equality relationships, in $n+m$ variables. The second equation above describes $m$ different equality relationships, also in a set of $n+m$ other variables. We know in order to solve the first equation we have to set $m$ of the variables to 0, and likewise we need to set $n$ of the variables in the second to 0. All that remains is to test the final 2 equations are true.

Lets take this example:

$
\begin{array}{c|cc}
\text{} & \text{X} & \text{Y} \\
\hline
\text{X} & 3,1 & 1,0 \\
\text{Y} & 0,0 & 2,2 \\
\end{array}
$

Below we iterate through all combinations. Note, that the first equation is encoded as 
$\big[U_1, I\big] \big(\begin{array}{c}s_2 \\r_1 \\\end{array}\big)$ and likewise for the second.